# PDF Tables 2 Data

PDFs often contain useful data tables of interest that you want to analyze. Thus it would be useful if you could extract it in an easy and straightforward manner. Copy and paste is an option, of course, except in the following scenarios:
1. Avoiding the formatting nightmare trying to rearrange everything into the correct format
2. If there are a lot of pages to deal with
3. If you have time on your hand and want to automate it, to improve your Python skillset

**Tip:** If you have another option to collect the same data, I would highly recommend that route first. Extracting data from PDF should be your last resort.

**Why?**  Tim Arnold explains briefly in his [blog on *Manipulating PDFs with Python*](https://www.binpress.com/tutorial/manipulating-pdfs-with-python/167).

> PDF documents are beautiful things, but that beauty is often only skin deep. Inside, they might have any number of structures that are difficult to understand and exasperating to get at. The PDF reference specification (ISO 32000-1) provides rules, but it is programmers who follow them, and they, like all programmers, are a creative bunch....That means that in the end, a beautiful PDF document is really meant to be read and its internals are not to be messed with....Chances are, now that is is inside the PDF, it is just a bunch of lines and numbers with no connection to its former structure of cells, formats, and headings.

If you don't have options, this is where Tabula comes in. *It's not perfect but its pretty darn good.* We'll see some of its imperfections along the way.

# <font color='darkblue'>Tabula</font>
Tabula is a tool for liberating data tables locked inside a pdf file. Its written in *Java* with bindings to *R, Ruby, Node.js and Python*. Its supported on Windows, Mac, and Linux. There is also a [stand alone version](http://tabula.technology/) for non-programmers.

**Note:** Tabula only works on text-based PDFs, not scanned documents.

For data analysts/scientists, output is exportable as dataframes which is awesome for *Python or R*. The workshop is in Python but the same function arguments should apply in the other languages.

## Requirements
- Java 7 or 8
- `pandas` (for Python)
- photo editor like Photoshop or paint.net (optional but highly recommended)

## Installation

`pip install tabula-py`

This notebook was created using `tabula-py v1.0.0`

Here is the Github page: https://github.com/chezou/tabula-py

## How to Use

Import the package

In [1]:
from tabula import read_pdf

Here is the [pdf about car crashes in Michigan](http://publications.michigantrafficcrashfacts.org/2015/Historical_5year.pdf) we will be using. Let's look at the table of interest.

In [ ]:
url = 'http://publications.michigantrafficcrashfacts.org/2015/Historical_5year.pdf'

## Table Extraction

To extract a table of interest, the main requirements are
1. Specifying the bounding box coordinates (in units of points)
2. Some other keyword arguments

#### Page 1 Top Table

In [ ]:
left, top, width, height = 26,73,560,240
pg1_top = read_pdf(url, pages=1, lattice=True, area=(top,left,top+height,left+width))
pg1_top

Voilà! Data in a format we can instantly use.

Let's check the data types of the columns.

In [ ]:
pg1_top.dtypes

Looks correct. Let's go through the steps of how we can extract the bounding box coordinates.

### Instructions for extracting coordinates
1. Take a screenshot containing the full pdf page of interest
2. Copy and paste into photo editor
3. Crop pdf page
4. Resize image. Set page ppi=72 and size to 8.5"x11"
5. Use rectangle select tool to outline table  
6. Read off rectangle coordinates

**Note:** 1pt = 1px when ppi=72 (72pt = 1 inch)

**Tip:** For those on Mac OS X, you can use the Preview app instead of a photo editor. It seems much simpler and straightforward than on Windows. Instructions located at github page near the bottom. https://github.com/chezou/tabula-py

## Exercise

Let's practice getting the bounding box coordinates. Extract the table at the bottom of page 1.


#### Page 1 Bottom Table <font color='white'>Answer: 50,576,512,70</font>

In [ ]:
left, top, width, height = 0,0,0,0
pg1_bottom = read_pdf(url, pages=1, lattice=True, area=(top,left,top+height,left+width))
pg1_bottom

**Note:** Tabula has trouble with the capitalization of letters sometimes. At least it got the letters correct.

### Page 4

This page has a table with merged cells.

In [ ]:
url = 'http://publications.michigantrafficcrashfacts.org/2015/Historical_5year.pdf'
left, top, width, height = 25,67,338,569
pg4 = read_pdf(url, pages=4, lattice=True, area=(top,left,top+height,left+width))
pg4

**Note:** Tabula has a mind of its own on which column to put merge cells in. In this case, I would have put it in the "HOLIDAY PERIOD" column.

In [ ]:
pg4.dtypes

Notice that all the entries in the table are strings or text. Tabula handled this one gracefully.

### Side Text

Let's see how it handles the text in the merged cell.

In [ ]:
left, top, width, height = 362,68,226,569
df4 = read_pdf(url, pages=4, lattice=True, area=(top,left,top+height,left+width))
df4

In [ ]:
from pprint import pprint as prettyprint
prettyprint(df4.iat[0,0])

## Table spanning Multiple Pages <font color='white'>50,53,500,680</font>

Here is another [pdf containing tax credits from the Keystone Innovation Zone (KIZ) Program in Pennsylvania](https://dced.pa.gov/download/2016-kiz-tax-credit-awards/?wpdmdl=69554) a client came to us with. It consists of one table broken up over multiple pages. We use the fact (or hope) that the tables are formatted approximately the same on each page. We can use the same cutoff area for all the pages and combine them into one. Just pass a list to the `pages` keyword argument.

## Exercise

In [ ]:
url = 'http://dced.pa.gov/download/2016-kiz-tax-credit-awards/?wpdmdl=69554'
left, top, width, height = 0,0,0,0
multiple_pages = read_pdf(url, pages=[1,2,3,4,5], lattice=True, area=(top,left,top+height,left+width))
multiple_pages

**Note:** It wasn't smart enough to filter out blank lines (although that might not be a desired feature).

Okay, so I've been holding out some info from you. If the table is the only thing in the page, then you don't need the `area` keyword argument. Try the example above without it.

<font color="white">Okay, what I said is not entirely true either. Sometimes, it will still work if there are other items on the page. Try the page 4 example above without the area keyword argument</font>

### Lattice False Option <font color='white'>33,94,536,461</font>

So far, we've only set the keyword argument `Lattice=True`. Seems like this should always be the case if we are working with tables, no?  Seems like it should not even be an option or at least the default (which it is not). Here is a case where we set `Lattice=False`.

### Exercise

In [ ]:
filename = 'summary_of_fees_collected.pdf'
left, top, width, height = 0,0,0,0
table = read_pdf(filename, pages=[2], lattice=False, area=(top,left,top+height,left+width), pandas_options={'thousands':','})
table

In [ ]:
table.dtypes

Rerun the two code blocks above again for pages 2,3. What changed? What is going on with Page 3 to cause that change? Hint below if needed.

<font color='white'>**Hint:** Look at Wexford County</font>

**Note:** A slight imperfection in the pdf can mess Tabula up. 

Luckily, Python string formatting can take care of this issue.

## Edge Cases (Pun Intended)

Here is a pdf of two tables split over multiple pages. However, unlike the previous example we saw, the tables were not carefully split apart. This can cause some issues.

In [ ]:
filename = 'workshop_registration.pdf'
left, top, width, height = 67,158,530,600
pg1 = read_pdf(filename, pages=1, lattice=True, area=(top,left,top+height,left+width))
pg1

First page provided no issues. Let's move on to the second page and try to extract the top and bottom tables.

### Exercise

In [ ]:
left, top, width, height = 0,0,0,0
pg2a = read_pdf(filename, pages=2, lattice=True, area=(top,left,top+height,left+width))
pg2a

In [ ]:
left, top, width, height = 0,0,0,0
pg2b = read_pdf(filename, pages=2, lattice=True, area=(top,left,top+height,left+width))
pg2b

Notice that we are missing some people. Why?

Sometimes we can cheat the system by being creative with our bounding box. *This is more art than science.*

In [ ]:
left, top, width, height = 0,0,0,0
pg2top = read_pdf(filename, pages=2, lattice=True, area=(top,left,top+height,left+width))
pg2top

We can use `pandas_options` keyword argument to clean up the header.<font color="white">69,27,485,43</font>

In [ ]:
pg2top = read_pdf(filename, pages=2, lattice=True, area=(top,left,top+height,left+width), pandas_options={'header':None})
pg2top

Creative bounding boxes won't work for the bottom table though (or at least I wasn't able to get the last row).

In [ ]:
left, top, width, height = 69,134,485,623
pg2bottom = read_pdf(filename, pages=2, lattice=True, area=(top,left,top+height,left+width))
pg2bottom.tail(5)

**Note:** Tabula has troubles with tables that are missing edges.

## Mining Paragraphs or Free Text

Page 1 of this [pdf on County Crash rankings](http://publications.michigantrafficcrashfacts.org/2015/CountyRanking.pdf) has no tables. Let's see how it does with a page of all text laid out in paragraph format.

In [ ]:
url = r'http://publications.michigantrafficcrashfacts.org/2015/CountyRanking.pdf'
left, top, width, height = 0,0,612,717
txt = read_pdf(url, pages=1, lattice=False, area=(top,left,top+height,left+width))
txt

Tabula fails in this regard. It doesn't work on everything, unfortunately. 

**Tip:** Tabula might not be the best tool if you are doing text mining. Time to use another tool.

## PDF Miner Example

Now let's switch to the other jupyter notebook to see how in general pdf text extraction is done.

# More Exercises

A. Extract the table on crashes regarding **Day of the Week** from the pdf below

In [ ]:
url = r'http://publications.michigantrafficcrashfacts.org/2016/Crash.pdf'

B. Extract the table on page 9 from the pdf below

In [ ]:
filename = '2013-02-005-v1.pdf'

C. Extract the 5 rows in tables on page 3 from the pdf below

In [ ]:
filename = '2013-02-005-v1.pdf'

D. Extract the table on page 3 from the pdf below

In [ ]:
filename = 'MDOT_fastfacts02-2011_345554_7.pdf'

E. Extract the bottom table including the last row from page 2 of the pdf below. Okay, I know we've already tried it but I wasn't being entirely truthful again. The solution is not to get the perfect bounding box coordinates. Think about the other keyword arguments we've learned today and think outside the box (pun intended).

In [ ]:
filename = 'workshop_registration.pdf'